In [945]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize 
from pandas import read_csv
import logging
from functools import reduce
import csv
from collections import defaultdict

## Configuration 

In [946]:
hi_df = pd.DataFrame()
hi_df = pd.read_csv( 'file/sample_output/healthinformation.csv') 
#data_df = data_df.to_dict(orient='records')
hi_df

,_id,_index_map,formid,answers.height,answers.bloodtype,answers.identification.number.code,answers.identification.type,answers.identification.number.pin,answers.income,answers.bp1_systolic
0,aQMjmsDcwMbPs5NyhqQPUZ,NaN,aQMjmsDcwMbPs5NyhqQPUZ,16,A,1033.2,SSS,201,60.0,120
1,DexsDsw39ss5NyhqQPUP,NaN,DexsDsw39ss5NyhqQPUP,23,B,2033.2,SSS,333,80.0,201


In [947]:
def set_for_keys(my_dict, key_arr, val):

    current = my_dict
    for i in range(len(key_arr)):
        key = key_arr[i]
        if key not in current:
            if i==len(key_arr)-1:
                current[key] = val
            else:
                current[key] = {}
        else:
            if type(current[key]) is not dict:
                print("not compatible with key structure requested")
                raise ValueError("Dictionary key already occupied")

        current = current[key]

    return my_dict

def to_formatted_json(df, sep="."):
    result = []
    for _, row in df.iterrows():
        parsed_row = {}
        for idx, val in row.iteritems():
            keys = idx.split(sep)
            
            parsed_row = set_for_keys(parsed_row, keys, val)
        
            print(parsed_row)
        result.append(parsed_row)
        #print(parsed_row)
        print('-----parsed_row-----')
    
    return result

def to_formatted_json_test(df, sep="."):
    result = []
    for _, row in df.iterrows():
        parsed_row = {}
        for idx, val in row.iteritems():
            keys = idx.split(sep)
            
            parsed_row = set_for_keys(parsed_row, keys, val)
            
            print('keys: ' , keys)
            print('values: ' , val)
            print('idx: ' , idx)
            print('----------')
            #print('parse_row: ',parsed_row)
        print(parsed_row)
        print('-----parsed_row-----')
        result.append(parsed_row)
        
    
    return result

#Where df was parsed from json-dict using json_normalize

In [948]:
#hi_df = to_formatted_json(data_df, sep=".")
#hi_df

## File loader

In [949]:
fdh_df  = pd.DataFrame()
fdh_df  = pd.read_csv( 'file/sample_output/answers.familydiseasehistory.csv') 
#data_df = data_df.to_dict(orient='records')
fdh_df = fdh_df.sort_values(['_id','_index_map'])
fdh_df

,_id,_index_map,answers.familydiseasehistory
3,DexsDsw39ss5NyhqQPUP,0,Head Cancer
4,DexsDsw39ss5NyhqQPUP,1,Stomach Cancer
0,aQMjmsDcwMbPs5NyhqQPUZ,0,"Respiratory ,disease"
1,aQMjmsDcwMbPs5NyhqQPUZ,1,Brain Cancer
2,aQMjmsDcwMbPs5NyhqQPUZ,2,Stomach Cancer


## GROUP BY IF FILE IS ALL ARRAY

In [950]:
fdh_df = fdh_df.groupby('_id').agg(lambda x: x.tolist())
fdh_df = fdh_df.reset_index()
fdh_df

,_id,_index_map,answers.familydiseasehistory
0,DexsDsw39ss5NyhqQPUP,"[0, 1]","[Head Cancer, Stomach Cancer]"
1,aQMjmsDcwMbPs5NyhqQPUZ,"[0, 1, 2]","[Respiratory ,disease, Brain Cancer, Stomach C..."


In [951]:
fdh_df = fdh_df.drop(['_index_map'], axis=1)
fdh_df

,_id,answers.familydiseasehistory
0,DexsDsw39ss5NyhqQPUP,"[Head Cancer, Stomach Cancer]"
1,aQMjmsDcwMbPs5NyhqQPUZ,"[Respiratory ,disease, Brain Cancer, Stomach C..."


In [954]:
merge_join = pd.merge(hi_df,fdh_df, left_on='_id', right_on='_id', how='left')
merge_join = merge_join.drop(['_id'], axis=1).drop(['_index_map'], axis=1).replace(np.nan,'')
merge_join


,formid,answers.height,answers.bloodtype,answers.identification.number.code,answers.identification.type,answers.identification.number.pin,answers.income,answers.bp1_systolic,answers.familydiseasehistory
0,aQMjmsDcwMbPs5NyhqQPUZ,16,A,1033.2,SSS,201,60.0,120,"[Respiratory ,disease, Brain Cancer, Stomach C..."
1,DexsDsw39ss5NyhqQPUP,23,B,2033.2,SSS,333,80.0,201,"[Head Cancer, Stomach Cancer]"


In [955]:
json_result = to_formatted_json(merge_join)

{'formid': 'aQMjmsDcwMbPs5NyhqQPUZ'}
{'formid': 'aQMjmsDcwMbPs5NyhqQPUZ', 'answers': {'height': 16}}
{'formid': 'aQMjmsDcwMbPs5NyhqQPUZ', 'answers': {'height': 16, 'bloodtype': 'A'}}
{'formid': 'aQMjmsDcwMbPs5NyhqQPUZ', 'answers': {'height': 16, 'bloodtype': 'A', 'identification': {'number': {'code': 1033.2}}}}
{'formid': 'aQMjmsDcwMbPs5NyhqQPUZ', 'answers': {'height': 16, 'bloodtype': 'A', 'identification': {'number': {'code': 1033.2}, 'type': 'SSS'}}}
{'formid': 'aQMjmsDcwMbPs5NyhqQPUZ', 'answers': {'height': 16, 'bloodtype': 'A', 'identification': {'number': {'code': 1033.2, 'pin': 201}, 'type': 'SSS'}}}
{'formid': 'aQMjmsDcwMbPs5NyhqQPUZ', 'answers': {'height': 16, 'bloodtype': 'A', 'identification': {'number': {'code': 1033.2, 'pin': 201}, 'type': 'SSS'}, 'income': 60.0}}
{'formid': 'aQMjmsDcwMbPs5NyhqQPUZ', 'answers': {'height': 16, 'bloodtype': 'A', 'identification': {'number': {'code': 1033.2, 'pin': 201}, 'type': 'SSS'}, 'income': 60.0, 'bp1_systolic': 120}}
{'formid': 'aQMjms

In [956]:
json.dumps(json_result)

'[{"formid": "aQMjmsDcwMbPs5NyhqQPUZ", "answers": {"height": 16, "bloodtype": "A", "identification": {"number": {"code": 1033.2, "pin": 201}, "type": "SSS"}, "income": 60.0, "bp1_systolic": 120, "familydiseasehistory": ["Respiratory ,disease", "Brain Cancer", "Stomach Cancer"]}}, {"formid": "DexsDsw39ss5NyhqQPUP", "answers": {"height": 23, "bloodtype": "B", "identification": {"number": {"code": 2033.2, "pin": 333}, "type": "SSS"}, "income": 80.0, "bp1_systolic": 201, "familydiseasehistory": ["Head Cancer", "Stomach Cancer"]}}]'

## JOIN SHOULD BE IN THE PERSISTENT DATABASE ALREADY

In [957]:
add_df  = pd.DataFrame()
add_df  = pd.read_csv( 'file/sample_output/answers.contactnumber.csv') 
#add_df = add_df.sort_values(['_id','_index_map'])
add_df

,_id,_index_map,answers.contactnumber.type
0,aQMjmsDcwMbPs5NyhqQPUZ,0,Landline
1,aQMjmsDcwMbPs5NyhqQPUZ,1,Mobile
2,DexsDsw39ss5NyhqQPUP,0,Landline
3,DexsDsw39ss5NyhqQPUP,1,Mobile


In [958]:
add_street_df  = pd.DataFrame()
add_street_df = pd.read_csv( 'file/sample_output/answers.contactnumber.ext.csv') 
add_street_df = add_street_df.sort_values(['_id','_index_map'])
#add_street_df = add_street_df.drop(['_id'], axis=1).drop(['_index_map'], axis=1).replace(np.nan,'')
add_street_df = add_street_df.drop(['_index_map'], axis=1).replace(np.nan,'')
add_street_df 

,_id,answers.contactnumber.ext.number,answers.contactnumber.ext.trunkline
3,DexsDsw39ss5NyhqQPUP,9,Reception
4,DexsDsw39ss5NyhqQPUP,5,Operator
5,DexsDsw39ss5NyhqQPUP,23,Mobile Trunkline
0,aQMjmsDcwMbPs5NyhqQPUZ,2,Reception
1,aQMjmsDcwMbPs5NyhqQPUZ,2,Mobile Trunkline
2,aQMjmsDcwMbPs5NyhqQPUZ,3,Operator


In [959]:
test = to_formatted_json_test(add_street_df)
test

keys:  ['_id']
values:  DexsDsw39ss5NyhqQPUP
idx:  _id
----------
keys:  ['answers', 'contactnumber', 'ext', 'number']
values:  9
idx:  answers.contactnumber.ext.number
----------
keys:  ['answers', 'contactnumber', 'ext', 'trunkline']
values:  Reception
idx:  answers.contactnumber.ext.trunkline
----------
{'_id': 'DexsDsw39ss5NyhqQPUP', 'answers': {'contactnumber': {'ext': {'number': 9, 'trunkline': 'Reception'}}}}
-----parsed_row-----
keys:  ['_id']
values:  DexsDsw39ss5NyhqQPUP
idx:  _id
----------
keys:  ['answers', 'contactnumber', 'ext', 'number']
values:  5
idx:  answers.contactnumber.ext.number
----------
keys:  ['answers', 'contactnumber', 'ext', 'trunkline']
values:  Operator
idx:  answers.contactnumber.ext.trunkline
----------
{'_id': 'DexsDsw39ss5NyhqQPUP', 'answers': {'contactnumber': {'ext': {'number': 5, 'trunkline': 'Operator'}}}}
-----parsed_row-----
keys:  ['_id']
values:  DexsDsw39ss5NyhqQPUP
idx:  _id
----------
keys:  ['answers', 'contactnumber', 'ext', 'number']
v

[{'_id': 'DexsDsw39ss5NyhqQPUP',
  'answers': {'contactnumber': {'ext': {'number': 9,
     'trunkline': 'Reception'}}}},
 {'_id': 'DexsDsw39ss5NyhqQPUP',
  'answers': {'contactnumber': {'ext': {'number': 5,
     'trunkline': 'Operator'}}}},
 {'_id': 'DexsDsw39ss5NyhqQPUP',
  'answers': {'contactnumber': {'ext': {'number': 23,
     'trunkline': 'Mobile Trunkline'}}}},
 {'_id': 'aQMjmsDcwMbPs5NyhqQPUZ',
  'answers': {'contactnumber': {'ext': {'number': 2,
     'trunkline': 'Reception'}}}},
 {'_id': 'aQMjmsDcwMbPs5NyhqQPUZ',
  'answers': {'contactnumber': {'ext': {'number': 2,
     'trunkline': 'Mobile Trunkline'}}}},
 {'_id': 'aQMjmsDcwMbPs5NyhqQPUZ',
  'answers': {'contactnumber': {'ext': {'number': 3,
     'trunkline': 'Operator'}}}}]

## Convert object into array of object

In [960]:

        
fofo = {}
print('-----')
fofo = {'number': 233, 'trunkline': 'Reception'}
test[0]['answers']['contactnumber']['ext'] = [test[0]['answers']['contactnumber']['ext']]
test[0]
test[0]['answers']['contactnumber']['ext'].append(fofo)
test[0]


-----


{'_id': 'DexsDsw39ss5NyhqQPUP',
 'answers': {'contactnumber': {'ext': [{'number': 9, 'trunkline': 'Reception'},
    {'number': 233, 'trunkline': 'Reception'}]}}}

In [961]:
json.dumps(test[0])

'{"_id": "DexsDsw39ss5NyhqQPUP", "answers": {"contactnumber": {"ext": [{"number": 9, "trunkline": "Reception"}, {"number": 233, "trunkline": "Reception"}]}}}'